In [1]:
#importing libriries
import pandas as pd
import numpy as np
import re

In [2]:
#importing data set
df = pd.read_csv('/Users/joaootaviomeirellesratton/Desktop/Ironhack/Labs_mycodes/Projects/Week_2/attacks.csv', sep=(','), encoding=('latin-1'))

In [3]:
#checking data set
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


## Data Cleaning

In [4]:
# Dropping entire rows with null values above 85%

df.drop(df[df.isnull().mean(axis=1) >= 0.85].index, inplace=True)

### 1) 'Year' Treatment

In [5]:
# Transforming values to int and classifying nan
df['Year'] = df['Year'].apply(lambda x: int(x) if x > 0 else 'N/A')

### 2) 'Country' Treatment

In [6]:
# Removing blanks
df['Country']= df['Country'].str.strip()

In [7]:
# Checking countries with the most shark attack records
country_share = df['Country'].value_counts(normalize=True).sort_values(ascending=False).mul(100).round(2).astype('str') + '%'
country_share

USA                       35.65%
AUSTRALIA                  21.4%
SOUTH AFRICA               9.26%
PAPUA NEW GUINEA           2.14%
NEW ZEALAND                2.05%
                           ...  
RED SEA / INDIAN OCEAN     0.02%
ST. MARTIN                 0.02%
MID-PACIFC OCEAN           0.02%
MALDIVE ISLANDS            0.02%
NORTH SEA                  0.02%
Name: Country, Length: 204, dtype: object

In [8]:
# Array to list
y = df['Country'].unique().tolist()

In [9]:
# Sort list of countries
z = [str(x) for x in y]
z.sort()

In [10]:
df['Country_new'] = df['Country']

In [11]:
mask = (df['Country_new'] != 'USA') & (df['Country_new'] != 'AUSTRALIA')

df['Country_new'][mask] = 'OTHER'

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
df['Country_new'].unique()

array(['USA', 'AUSTRALIA', 'OTHER'], dtype=object)

### 3) 'Area' Treatment

In [13]:
# Changing nan to unknown area
df['Area'] = df['Area'].fillna('unknown')

In [14]:
# Standardizing strings, removing blanks and setting to lower case
df['Area'] = df['Area'].apply(lambda x: str(x).strip().lower())

In [15]:
# Group of classified oceans by area
ocean_area = {
'north_pacific' : ['california', 'hawaii', 'oregon', 'palmyra atoll', 'johnston atoll', 'alaska', 'midway atoll', 'guam', 'wake island'],
'north_atlantic' : ['georgia', 'florida', 'south carolina', 'new york', 'massachusetts', 'new jersey', 'washington', 'north carolina', 
    'maryland', 'delaware', 'maine', 'virginia', 'north carolina', 'rhode island', 'north & south carolina','connecticut', 'east coast'],
'other_locations' : ['kentucky', 'new mexico', 'missouri', 'pennsylvania', 'illinois'],
'gulf_caribean' : ['texas','alabama', 'louisiana', 'puerto rico', 'us virgin islands', 'cuba','mississipi'],
'south_pacific': ['new south wales','queensland','victoria','south australia','tasmania','torres strait','northern territory', 'norfolk island'],
'indian' : ['westerm australia','western australia', 'territory of cocos (keeling) islands']
}

In [16]:
# Defining ocean based on area
def ocean_location(x):
    for i in range(len(ocean_area)):
        if x in list(ocean_area.values())[i]:
            return list(ocean_area.keys())[i]

In [17]:
# Defining ocean based on area
df['Ocean'] = df['Area'].apply(ocean_location)

### 4) 'Species' Treatment

In [18]:
# Changing nan to unknown species
df['Species '] = df['Species '].fillna('Unknown')

In [19]:
# Standardizing strings, removing blanks and setting to lower case
df['Species '] = df['Species '].apply(lambda x: str(x).strip().lower())

In [20]:
# Creating a new collumn that will contain treated strings
df['species_treated'] = df['Species ']

In [21]:
# Classifying species
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*white.*', 'white shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*tiger.*', 'tiger shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*bull.*', 'bull shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*lemon.*', 'lemon shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*grey.*', 'grey reef shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*tawn[e]y.*', 'tawny nurse shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*spinner.*', 'spinner shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*blacktip.*', 'blacktip shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*hammerhead.*', 'hammerhead shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*wobbegong.*', 'wobbegong shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*cookie.*', 'cookiecutter shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*seven[ -]gill.*', 'seven_gill shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*sevengill.*', 'seven_gill shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*reef.*', 'reef shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*dogfish.*', 'dogfish shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*whaler.*', 'whaler shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*mako.*', 'mako shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*nurse.*', 'nurse shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*blue.*', 'blue shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*sand.*', 'sand shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*galapagos.*', 'galapagos shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*salmon.*', 'salmon shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*cat.*', 'cat shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*raggedtooth.*', 'raggedtooth shark', x))
df['species_treated'] = df['species_treated'].apply(lambda x: re.sub('.*carpet.*', 'carpet shark', x))


In [22]:
# Group of classified species
shark_species = ['blacktip shark','blue shark','bull shark','carpet shark','cat shark',
'cookiecutter shark','dogfish shark','grey reef shark','hammerhead shark','lemon shark',
'mako shark','nurse shark','raggedtooth shark','reef shark','salmon shark','sand sharkgalapagos shark',
'seven_gill shark','spinner shark','tawny nurse shark','tiger shark','whaler shark','white shark',
'wobbegong shark']

In [23]:
# Species/string not in defined grouped will be classified as n/a
def check_species(x):
    if x not in shark_species:
        x = 'n/a'
    else:
        x
    return x

In [24]:
# Species/string not in defined grouped will be classified as n/a
df['species_treated'] = df['species_treated'].apply(lambda x: check_species(x))

## Analysis - Is there a difference among shark species attacks according to the ocean? 

In [25]:
# Filtering countries (USA and Australia, together they account for over 55% of reported cases), removing invalid cases and undefined shark species
df_analysis = df.query('(Country == "AUSTRALIA") | (Country == "USA")').query('Type != "Invalid"').query('species_treated != "n/a"')

In [26]:
# Counting occurrences by Country/Ocean/Shark Species
df_analysis.groupby(by=['Country', 'Ocean','species_treated']).agg(case_count = ('Country', 'count')).reset_index()

,Country,Ocean,species_treated,case_count
0,AUSTRALIA,indian,blacktip shark,1
1,AUSTRALIA,indian,blue shark,1
2,AUSTRALIA,indian,bull shark,3
3,AUSTRALIA,indian,carpet shark,5
4,AUSTRALIA,indian,hammerhead shark,1
...,...,...,...,...
57,USA,north_pacific,tiger shark,86
58,USA,north_pacific,white shark,184
59,USA,other_locations,cat shark,1
60,USA,other_locations,nurse shark,1


In [27]:
# Exporting to csv
df_analysis.to_csv('../shark_ocean_analysis.csv')

In [29]:
y = df.query('Area == "new mexico"')

In [31]:
list(y['href'])

['http://sharkattackfile.net/spreadsheets/pdf_directory/2005.03.12-Pitts.pdf']